## Technical Assessment
### Internship- Data Analyst
### Part 1:
### Use Python
Extract the maximum job posts through LinkedIn web scraping. Please keep these points in mind
before extraction
1. Job Location: Worldwide except USA and Canada
2. Job Type- Full time and Contract
3. Work format- Remote
4. Maximum information of company and job poster
5. Unique records of Company&Job_Location&Title&Job_Post_URL only
You may refer to the Link for reference.
NOTE- Please keep yourself logged out from LinkedIn while scraping.
### Part 2:
To make it scalable please sync the output to google sheet (Share property- Anyone with the link)
NOTE- Each run unique output should append to the old extracted output

In [ ]:
from selenium import webdriver
import time
import pandas as pd
import os

import gspread
from df2gspread import df2gspread as d2g
import google.auth

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver_path = 'B:\Downloads\chromedriver_win32\chromedriver.exe'

#Note: For filtering out full time and contract job types and remote work format,
#I applied the filters on the linkedin jobs website and then copied the url

url1='https://www.linkedin.com/jobs/jobs-in-worldwide?keywords=&location=Worldwide&locationId=&geoId=92000000&f_TPR=&f_JT=F%2CC&f_WT=2&position=1&pageNum=0'

In [ ]:
service = webdriver.chrome.service.Service(driver_path)
service.start()
driver = webdriver.Remote(service.service_url)
driver.get(url1)
driver.implicitly_wait(10)

In [ ]:
#Find number of job listings

y=driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text.rstrip("+").replace(",","")
n=int(y)
n

In [ ]:
#Loop to scroll through all jobs and click on see more jobs button for infinite scrolling

i = 2
while i <3: #Condition can be changed to scrap larger dataset
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    
    try:
        send=driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
        driver.execute_script("arguments[0].click();", send)   
        time.sleep(3)
    
    except:
        pass
        time.sleep(5)
         

In [ ]:
#Create empty lists to store data

companyname= []
titlename= []
location= []

In [ ]:
url= []

In [ ]:
#Find company name and append it to the blank list

try:
    for i in range(n):
        company=driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')[i].text
        companyname.append(company)
        
except IndexError:
    print("no")

In [ ]:
len(companyname)

In [ ]:
#Find title name and append it to the blank list

try:
    for i in range(n):
        title=driver.find_elements(By.CLASS_NAME, 'base-search-card__title')[i].text
        titlename.append(title)
        
except IndexError:
    print("no")

In [ ]:
len(titlename)

In [ ]:
#Find location and append it to the blank list

try:
    for i in range(n):
        loc=driver.find_elements(By.CLASS_NAME, 'job-search-card__location')[i].text
        location.append(loc)
        
except IndexError:
    print("no")

In [ ]:
len(location)

In [ ]:
#Find post url and append it to the blank list

link=driver.find_elements(By.CLASS_NAME, 'base-card__full-link')
for k in link:
    url.append(k.get_attribute('href'))

In [ ]:
len(url)

In [ ]:
#Create dataframe

data = {"Company": companyname, "Title": titlename, "Location": location, "Post URL": url}

jobsdf = pd.DataFrame(data)

In [ ]:
jobsdf

In [ ]:
#Filtering jobs according to location i.e. worldwide except USA and Canada

index_labels = jobsdf[(jobsdf['Location'].str.contains('United States')) | (jobsdf['Location'].str.contains('Canada'))].index

jobsdf.drop(index_labels, inplace=True)

In [ ]:
jobsdf

# Part 2 

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials


def create_connection(service_file):
    client = None
    scope = [
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/drive.file",
    ]
    try:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            service_file, scope
        )
        client = gspread.authorize(credentials)
        print("Connection established successfully...")
    except Exception as e:
        print(e)
    return client


service = "jobscraping-410904-f0a2fc80b073.json"  
client = create_connection(service)

In [ ]:
def create_google_sheet(client, sheet_name):
    google_sheet = None
    try:
        google_sheet = client.create(sheet_name)
        google_sheet.share(
            "nitindhiman112780@gmail.com",  
            perm_type="user",
            role="writer",
        )
        print("Google Sheet created successfully...")
    except Exception as e:
        print(e)
    return google_sheet


service = "jobscraping-410904-f0a2fc80b073.json"  
client = create_connection(service)

sheet_name = "Scraped_Jobs"  
google_sheet = create_google_sheet(client, sheet_name)

In [ ]:
def export_dataframe_to_google_sheet(worksheet, dataframe):
    try:
        worksheet.update(
            [dataframe.columns.values.tolist()] + dataframe.values.tolist()
        )
        print("DataFrame exported successfully...")
    except Exception as e:
        print(e)

worksheet = google_sheet.get_worksheet(0)
export_dataframe_to_google_sheet(worksheet, jobsdf)